In [20]:
# https://www.energate-messenger.de/markt/strom/preisgruppe/174994/eex-phelix-de-futures-base

import time
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import glob

In [21]:
# get the content of the website
page = requests.get("https://www.energate-messenger.de/markt/strom/preisgruppe/174994/eex-phelix-de-futures-base")
html = page.content

In [22]:
# parse the html and save it into a BeautifulSoup instance
bs = BeautifulSoup(html, 'html.parser')

In [23]:
bs.find('thead').find_all('th')

[<th colspan="2">Preisreihe</th>,
 <th>Datum</th>,
 <th>Preis</th>,
 <th>Preis Tief</th>,
 <th>Preis Hoch</th>,
 <th>Umsatz</th>,
 <th>Open Interest</th>,
 <th>No. of Trades</th>]

In [24]:
pricetable = bs.find('tbody')
pricetable.find_all('td')

[<td class="type">Jahr</td>,
 <td><a href="/markt/strom/preise/175102/eex-phelix-de-baseload-cal-2023">Jahr 2023</a></td>,
 <td class="date">29.06.2022</td>,
 <td class="numeric">281,35</td>,
 <td class="numeric">273,00</td>,
 <td class="numeric">285,00</td>,
 <td class="numeric">1.384.080</td>,
 <td class="numeric">80.456</td>,
 <td class="numeric">131</td>,
 <td class="type">Jahr</td>,
 <td><a href="/markt/strom/preise/175103/eex-phelix-de-baseload-cal-2024">Jahr 2024</a></td>,
 <td class="date">29.06.2022</td>,
 <td class="numeric">174,05</td>,
 <td class="numeric">170,00</td>,
 <td class="numeric">176,00</td>,
 <td class="numeric">535.824</td>,
 <td class="numeric">20.316</td>,
 <td class="numeric">48</td>,
 <td class="type">Jahr</td>,
 <td><a href="/markt/strom/preise/175104/eex-phelix-de-baseload-cal-2025">Jahr 2025</a></td>,
 <td class="date">29.06.2022</td>,
 <td class="numeric">142,48</td>,
 <td class="numeric">140,75</td>,
 <td class="numeric">142,50</td>,
 <td class="numeric

In [25]:
pricetable.find_all(class_="numeric")

[<td class="numeric">281,35</td>,
 <td class="numeric">273,00</td>,
 <td class="numeric">285,00</td>,
 <td class="numeric">1.384.080</td>,
 <td class="numeric">80.456</td>,
 <td class="numeric">131</td>,
 <td class="numeric">174,05</td>,
 <td class="numeric">170,00</td>,
 <td class="numeric">176,00</td>,
 <td class="numeric">535.824</td>,
 <td class="numeric">20.316</td>,
 <td class="numeric">48</td>,
 <td class="numeric">142,48</td>,
 <td class="numeric">140,75</td>,
 <td class="numeric">142,50</td>,
 <td class="numeric">122.640</td>,
 <td class="numeric">5.958</td>,
 <td class="numeric">12</td>,
 <td class="numeric">362,23</td>,
 <td class="numeric">353,50</td>,
 <td class="numeric">365,00</td>,
 <td class="numeric">340.186</td>,
 <td class="numeric">190.305</td>,
 <td class="numeric">97</td>,
 <td class="numeric">337,52</td>,
 <td class="numeric">325,00</td>,
 <td class="numeric">340,00</td>,
 <td class="numeric">198.628</td>,
 <td class="numeric">17.665</td>,
 <td class="numeric">6

---

## Lists

### PriceSeries
First we store the original header in a list. After we adapt our own formatting.

In [26]:
headers = pricetable.find_all('a')
headers_lst = [header.get_text() for header in headers]
headers_lst[::5]

['Jahr 2023',
 'Quartal 2/2023',
 'Quartal 3/2024',
 'September 2022',
 'Februar 2023']

In [27]:
headers_lst = [sub.replace('20', '') for sub in headers_lst]
headers_lst = [sub.replace('Jahr ', 'cal-') for sub in headers_lst]
headers_lst = [sub.replace('Quartal ', 'Q') for sub in headers_lst]
headers_lst = [sub.replace('/', '-') for sub in headers_lst]
headers_lst = [sub.replace('Januar ', 'M1-') for sub in headers_lst]
headers_lst = [sub.replace('Februar ', 'M2-') for sub in headers_lst]
headers_lst = [sub.replace('März ', 'M3-') for sub in headers_lst]
headers_lst = [sub.replace('Maerz ', 'M3-') for sub in headers_lst]
headers_lst = [sub.replace('April ', 'M4-') for sub in headers_lst]
headers_lst = [sub.replace('Mai ', 'M5-') for sub in headers_lst]
headers_lst = [sub.replace('Juni ', 'M6-') for sub in headers_lst]
headers_lst = [sub.replace('Juli ', 'M7-') for sub in headers_lst]
headers_lst = [sub.replace('August ', 'M8-') for sub in headers_lst]
headers_lst = [sub.replace('September ', 'M9-') for sub in headers_lst]
headers_lst = [sub.replace('Oktober ', 'M10-') for sub in headers_lst]
headers_lst = [sub.replace('November ', 'M11-') for sub in headers_lst]
headers_lst = [sub.replace('Dezember ', 'M12-') for sub in headers_lst]
headers_lst[::5]

['cal-23', 'Q2-23', 'Q3-24', 'M9-22', 'M2-23']

In [28]:
types = pricetable.find_all(class_='type')
types_lst = [typ.get_text() for typ in types]
types_lst[::5]

['Jahr', 'Quartal', 'Quartal', 'Monat', 'Monat']

In [29]:
dates = pricetable.find_all(class_="date")
dates_lst = [date.get_text() for date in dates]
dates_lst = [sub.replace('.', '/') for sub in dates_lst]
dates_lst[::5]

['29/06/2022', '29/06/2022', '29/06/2022', '29/06/2022', '29/06/2022']

### Numerics
Please note that in this list 6 columns are stored

In [30]:
numerics = pricetable.find_all(class_='numeric')
numerics_lst = [numeric.get_text() for numeric in numerics]
numerics_lst = ([sub.replace('.','') for sub in numerics_lst])
numerics_lst = ([sub.replace(',','.') for sub in numerics_lst])
numerics_lst[::5]

['281.35',
 '131',
 '20316',
 '122640',
 '365.00',
 '325.00',
 '246.38',
 '39',
 '6370',
 '97196',
 '257.00',
 '-',
 '132.01',
 '-',
 '172',
 '-',
 '290.00',
 '290.00',
 '333.79',
 '64',
 '1039',
 '-',
 '-',
 '-',
 '369.16',
 '-',
 '-']

In [31]:
def make_sublists(main_list, sublist_size):
    return [main_list[x:x+sublist_size]
            for x in range(0, len(main_list), sublist_size)]

In [32]:
def extract(lst, ii):
    return [item[ii] for item in lst]

In [33]:
df = pd.DataFrame(list(zip(headers_lst, types_lst, dates_lst)), columns=['PriceSeries', 'PriceType', 'Date'])

# Numeric Columns
hlist = ['Price', 'PriceLow', 'PriceHigh', 'Volumne', 'OpenInterest', 'TradesCount']
for i in range(len(hlist)):
    df[hlist[i]] = extract(make_sublists(numerics_lst, 6),i)
    
df

,PriceSeries,PriceType,Date,Price,PriceLow,PriceHigh,Volumne,OpenInterest,TradesCount
0,cal-23,Jahr,29/06/2022,281.35,273.00,285.00,1384080,80456,131
1,cal-24,Jahr,29/06/2022,174.05,170.00,176.00,535824,20316,48
2,cal-25,Jahr,29/06/2022,142.48,140.75,142.50,122640,5958,12
3,Q4-22,Quartal,29/06/2022,362.23,353.50,365.00,340186,190305,97
4,Q1-23,Quartal,29/06/2022,337.52,325.00,340.00,198628,17665,61
5,Q2-23,Quartal,29/06/2022,246.38,243.50,249.50,124488,6852,39
6,Q3-23,Quartal,29/06/2022,246.34,240.50,250.00,99360,6370,24
7,Q4-23,Quartal,29/06/2022,296.02,290.00,297.00,97196,6549,22
8,Q1-24,Quartal,29/06/2022,257.05,257.00,257.00,4366,682,2
9,Q2-24,Quartal,29/06/2022,134.63,-,-,-,533,-


### Combine old csv [optional]

In [34]:
if False:
    df_eex = pd.DataFrame()
    for filename in glob.iglob('eex*.csv'):
        #print(filename)
        df = pd.read_csv(filename)
        df_eex = pd.concat([df_eex, df])

    print(df_eex.info())

    headers_lst = df_eex['PriceSeries']
    headers_lst = [sub.replace('20', '') for sub in headers_lst]
    headers_lst = [sub.replace('Jahr ', 'cal-') for sub in headers_lst]
    headers_lst = [sub.replace('Quartal ', 'Q') for sub in headers_lst]
    headers_lst = [sub.replace('/', '-') for sub in headers_lst]
    headers_lst = [sub.replace('Januar ', 'M1-') for sub in headers_lst]
    headers_lst = [sub.replace('Februar ', 'M2-') for sub in headers_lst]
    headers_lst = [sub.replace('März ', 'M3-') for sub in headers_lst]
    headers_lst = [sub.replace('Maerz ', 'M3-') for sub in headers_lst]
    headers_lst = [sub.replace('April ', 'M4-') for sub in headers_lst]
    headers_lst = [sub.replace('Mai ', 'M5-') for sub in headers_lst]
    headers_lst = [sub.replace('Juni ', 'M6-') for sub in headers_lst]
    headers_lst = [sub.replace('Juli ', 'M7-') for sub in headers_lst]
    headers_lst = [sub.replace('August ', 'M8-') for sub in headers_lst]
    headers_lst = [sub.replace('September ', 'M9-') for sub in headers_lst]
    headers_lst = [sub.replace('Oktober ', 'M10-') for sub in headers_lst]
    headers_lst = [sub.replace('November ', 'M11-') for sub in headers_lst]
    headers_lst = [sub.replace('Dezember ', 'M12-') for sub in headers_lst]
    df_eex['PriceSeries'] = headers_lst


    dates_lst = df_eex['Date'] 
    dates_lst= [sub.replace('.', '/') for sub in dates_lst]
    df_eex['Date'] = dates_lst

    df_eex.to_csv('eex.csv', index=False)
    

### Combine all eex date with new

In [35]:
# Read in old eex
df_eex = pd.read_csv('eex.csv')
df_eex = pd.concat([df_eex, df])

In [36]:
# Remove duplicates and sort by date
df_eex.drop_duplicates(subset=['PriceSeries', 'Date'], keep='first', inplace=True)
df_eex.sort_values(by='Date', inplace=True)

In [37]:
df_eex

,PriceSeries,PriceType,Date,Price,PriceLow,PriceHigh,Volumne,OpenInterest,TradesCount
0,Q2-22,Quartal,01/03/2022,332.0,NaN,NaN,NaN,NaN,NaN
1,Q3-22,Quartal,01/03/2022,323.0,NaN,NaN,NaN,NaN,NaN
2,cal-23,Jahr,01/03/2022,155.92,NaN,NaN,NaN,NaN,NaN
3,Q4-22,Quartal,01/03/2022,317.5,NaN,NaN,NaN,NaN,NaN
12,cal-23,Jahr,01/04/2022,186.53,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
965,M11-22,Monat,31/05/2022,288.19,-,-,-,70,-
966,Q1-24,Quartal,31/05/2022,234.66,-,-,-,562,-
967,Q3-22,Quartal,31/05/2022,219.88,214.00,221.76,331200,199522,112
960,M10-22,Monat,31/05/2022,235.59,-,-,-,615,-


### Save to CSV

In [38]:
 df_eex.to_csv('eex.csv', index=False)